In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
class NeuronPopRecording: #Suppose to be one file
    metadata_datanames=['noNeurons', 'dt', 'totalTimesteps', 'neuronPopId']
    def __init__(self, neuronPop_id : int):
        self.neuronPop_id=neuronPop_id
    def append_metadata(self, metadata_list : list):
        self.metadata=metadata_list
    def append_spiketimes(self, N_index : int, spiketimes_list : list):
        setattr(self, "Neuron_%s" % (N_index), spiketimes_list)

class OutputRecordingList: #Compendium of all recordings
    def get(self):
        return self.__dict__
    def append(self, var : NeuronPopRecording):
        setattr(self, "NeuronPop_%s" % (var.neuronPop_id), var)



### File reading

In [9]:
#File reading here (files are always iterated on alphabetical order)
parsedOutput_string='_ParsedOutput.dat'

base_dir = os.getcwd()

neuronPop_files = [f for f in os.listdir(base_dir) if f.endswith(parsedOutput_string)]

['AnalyseData.Rmd',
 'hetero_analyser_mono_dendrite.ipynb',
 'LoadData.R',
 'ParsedFileReader.ipynb',
 'README.txt']

### Reading parsed spike times file (Neuron output)

In [7]:
#Here we use split = to get the name and the values, then split the second with commas to get the spike-time list
def read_all_output_files_into_object(neuronPop_files):
    all_recordingsST=OutputRecordingList()
    for i, file in enumerate(neuronPop_files):
        with open(file) as file:
            firstline=file.readlines()[0].rstrip() #The first line should be the metadata
            if not firstline.startswith('M'):
                print('Error with metadata reading')
                raise Exception
            metadata_list=firstline.split('=')[1].split(',')
            metadata_list[0]=int(metadata_list[0])
            metadata_list[1]=float(metadata_list[1])
            metadata_list[2]=int(metadata_list[2])
            metadata_list[3]=int(metadata_list[3])
            recording=NeuronPopRecording(metadata_list[3])
            recording.append_metadata(metadata_list)
            for line in file.readlines():
                if line.startswith('M'):
                    pass
                elif line.startswith('N'):
                    line=line.split('=')
                    n_index=int(line[0].split('_')[1])
                    spiketimes=list()
                    for i in line[1].split(','):
                        if i=='':
                            continue
                        else:
                            spiketimes.append(i)
                    recording.append_spiketimes(n_index, spiketimes)
                else:
                    print('Oversight error')
                    raise Exception
        all_recordingsST.append(recording)
    return all_recordingsST

{'NeuronPop_0': [0, 2, 4]}

In [ ]:
all_recordings=read_all_output_files_into_object(neuronPop_files)